In [1]:
import logging
from sentinelhub import DataCollection
from sentinelhub import SHConfig
from sentinel_api import sentinel_single_image, sentinel_territory
import numpy as np
from PIL import Image
import os

### Logging in sentinel account

In [2]:
logging.basicConfig(level=logging.INFO)

# Your client credentials
client_id = 'e0b127dc-fbb5-4151-8946-96d8728003c7'
client_secret = '4Xy2[Bc#?&L!xbha(XB6*%Xbry,BU-cZA0Dd9n,W'

config = SHConfig(instance_id='',
         sh_client_id = client_id,
         sh_client_secret = client_secret,
         sh_base_url='https://services.sentinel-hub.com',
         sh_token_url='https://services.sentinel-hub.com/oauth/token',)

### Download an image from Sentinel-hub API (fn: sentinel_single_image)

In [ ]:
# fn input
bbox = [58.50, 23.65, 58.60, 23.75]
timeline = ('2023-07-05', '2023-09-25')

# Parameters
mosaicking_order_modes = ['mostRecent', 'leastCC', 'leastRecent']
data_collection_modes = [DataCollection.SENTINEL2_L2A, DataCollection.SENTINEL2_L1C]

img = sentinel_single_image(bbox, timeline, config=config, data_collection=data_collection_modes[1], mosaicking_order = mosaicking_order_modes[0],
                            resolution=10, maxcc=0.8, img_size=(2500,2500), return_numpy=False, verbose=True, save_image=False, save_dir="sentinel-hub")
img.save(r"C:\Users\user2\Desktop\sentinel-3.png")
img

### Download images of a vast area from Sentinel-hub API (fn: sentinel_territory)

#### In disk

In [3]:
# fn inputs:
bbox_coords = [58.5, 23.6, 59, 24.1]
bbox_coords = [58.5, 23.6, 58.8, 23.9]
timeline = ('2023-07-05', '2023-09-25')

# Parameters
mosaicking_order_modes = ['mostRecent', 'leastCC', 'leastRecent']
data_collection_modes = [DataCollection.SENTINEL2_L2A, DataCollection.SENTINEL2_L1C]

final_image = sentinel_territory(bbox_coords, timeline, config, data_collection=data_collection_modes[1], mosaicking_order = 'mostRecent', maxcc=0.8,
                                resolution=5, img_size=(128, 128), save_concat_image=True, in_memory=False, temp_dir=r"sentinel-tmp",
                                concat_image_dir=r"sentinel-concat", concat_image_name="default", delete_temp=True)

Territory dimensions: 30,568m x 33,358m | Area: 1,020 km^2
Concatenated image will be in size of 14999 x 14999 p
There are 36 bboxes to download



dl from sentinel-hub | bbox: [58.75, 23.85, 58.8, 23.9]: 100%|█████████████████████████| 36/36 [00:41<00:00,  1.15s/it]

All images are downloaded into disk - concatenating images is in progress...
Done!
concatenated image saved in sentinel-concat


#### In memory

In [ ]:
# fn inputs
bbox_coords = [58.5, 23.6, 59, 24.1]
timeline = ('2023-07-05', '2023-09-25')

# Parameters
mosaicking_order_modes = ['mostRecent', 'leastCC', 'leastRecent']
data_collection_modes = [DataCollection.SENTINEL2_L2A, DataCollection.SENTINEL2_L1C]

img_array, final_image = sentinel_territory(bbox_coords, timeline, config, data_collection=data_collection_modes[1], mosaicking_order = 'mostRecent', maxcc=0.8,
                        resolution=10, save_images=False, data_folder="sentinel-hub", subfolder_name="default", img_size=(64, 64))

### Integrated Inference (sentinel and inference modules integration)

### Drafts